In [2]:
import numpy as np
np.set_printoptions(threshold=10000,suppress=True)
import pandas as pd
import warnings
import matplotlib
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [3]:
# 1. Chargement des données
df = pd.read_csv('iris.txt', sep='\t', header=None)
X = df.iloc[:, :4].values  
y = df.iloc[:, 4].values   

print("Shape de X:", X.shape)
print("Shape de y:", y.shape)
print("\n5 premières lignes de X:")
print(X[:5])
print("\n5 premières valeurs de y:")
print(y[:5])


Shape de X: (150, 4)
Shape de y: (150,)

5 premières lignes de X:
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]

5 premières valeurs de y:
[1 1 1 1 1]


In [5]:
# 2. Découpage stratifié en Apprentissage/Test (2/3 - 1/3)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=1/3,           # 1/3 pour test
    random_state=42,         
    stratify=y               )

print(f"\nTaille de l'apprentissage: {len(X_train)} échantillons")
print(f"Taille du test: {len(X_test)} échantillons")
print(f"Répartition des classes dans y_train: {np.bincount(y_train)}")
print(f"Répartition des classes dans y_test: {np.bincount(y_test)}")



Taille de l'apprentissage: 100 échantillons
Taille du test: 50 échantillons
Répartition des classes dans y_train: [ 0 34 33 33]
Répartition des classes dans y_test: [ 0 16 17 17]


In [ ]:
import numpy as np

class MultiClassPerceptron:

    def __init__(self, learning_rate=0.01, max_iter=1000):

        self.lr = learning_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None     

    def fit(self, X, y):

        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))
        
        self.weights = np.zeros((n_classes, n_features))
        self.bias = np.zeros(n_classes)

        for _ in range(self.max_iter):
            for idx, x_i in enumerate(X):
                # Prédiction
                linear_output = np.dot(self.weights, x_i) + self.bias
                y_pred = np.argmax(linear_output)
                # Mise à jour
                if y_pred != y[idx]:
                    self.weights[y[idx]] += self.lr * x_i
                    self.bias[y[idx]] += self.lr
                    self.weights[y_pred] -= self.lr * x_i
                    self.bias[y_pred] -= self.lr

    def predict(self, X):
        linear_output = np.dot(X, self.weights.T) + self.bias
        return np.argmax(linear_output, axis=1)

In [7]:
batch_size = 10 
nb_epochs = 5 
eta = 0.001 
hidden_size1 = 16  

# 2. Découpage stratifié en Apprentissage/Test (2/3 - 1/3)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print("Classes originales:", label_encoder.classes_)
print("Encodage:", dict(zip(label_encoder.classes_, range(len(label_encoder.classes_)))))

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, 
    test_size=1/3,           
    random_state=42,        
    stratify=y_encoded       
)

print(f"\nTaille de l'apprentissage: {len(X_train)} échantillons")
print(f"Taille du test: {len(X_test)} échantillons")
print(f"Répartition des classes dans y_train: {np.bincount(y_train)}")
print(f"Répartition des classes dans y_test: {np.bincount(y_test)}")

Classes originales: [1 2 3]
Encodage: {np.int64(1): 0, np.int64(2): 1, np.int64(3): 2}

Taille de l'apprentissage: 100 échantillons
Taille du test: 50 échantillons
Répartition des classes dans y_train: [34 33 33]
Répartition des classes dans y_test: [16 17 17]


In [8]:

print("3. PERCEPTRON MULTI-CLASSE")


# Entraînement du Perceptron
perceptron = MultiClassPerceptron(learning_rate=0.01, max_iter=1000)
perceptron.fit(X_train, y_train)

print("Entraînement terminé!")
print(f"Poids finaux shape: {perceptron.weights.shape}")
print(f"Biais finaux: {perceptron.bias}")

3. PERCEPTRON MULTI-CLASSE
Entraînement terminé!
Poids finaux shape: (3, 4)
Biais finaux: [ 0.31  2.21 -2.52]


In [ ]:
# 4. Évaluation des performances du Perceptron

print("4. ÉVALUATION DU PERCEPTRON")

y_pred_perceptron = perceptron.predict(X_test)

# Fonction pour calculer la matrice de confusion
def confusion_matrix(y_true, y_pred, n_classes):
    cm = np.zeros((n_classes, n_classes), dtype=int)
    for true, pred in zip(y_true, y_pred):
        cm[true, pred] += 1
    return cm

# Fonction pour calculer les métriques
def compute_metrics(y_true, y_pred, n_classes, class_names=None):
    cm = confusion_matrix(y_true, y_pred, n_classes)
    
    # Accuracy globale
    accuracy = np.sum(np.diag(cm)) / np.sum(cm)
    
    # Précision et Rappel par classe
    precision = np.zeros(n_classes)
    recall = np.zeros(n_classes)
    
    for i in range(n_classes):
        # Précision = VP / (VP + FP)
        if np.sum(cm[:, i]) > 0:
            precision[i] = cm[i, i] / np.sum(cm[:, i])
        else:
            precision[i] = 0
        
        # Rappel = VP / (VP + FN)
        if np.sum(cm[i, :]) > 0:
            recall[i] = cm[i, i] / np.sum(cm[i, :])
        else:
            recall[i] = 0
    
    return cm, accuracy, precision, recall

n_classes = len(np.unique(y_train))
cm_perceptron, acc_perceptron, prec_perceptron, rec_perceptron = compute_metrics(
    y_test, y_pred_perceptron, n_classes
)

print("\nMatrice de Confusion:")
print(cm_perceptron)
print(f"\nAccuracy globale: {acc_perceptron:.4f} ({acc_perceptron*100:.2f}%)")
print("\nMétriques par classe:")
for i in range(n_classes):
    class_name = label_encoder.classes_[i]
    print(f"Classe {i} ({class_name}): Précision = {prec_perceptron[i]:.4f}, Rappel = {rec_perceptron[i]:.4f}")


4. ÉVALUATION DU PERCEPTRON

Matrice de Confusion:
[[16  0  0]
 [ 0 17  0]
 [ 0  5 12]]

Accuracy globale: 0.9000 (90.00%)

Métriques par classe:
Classe 0 (1): Précision = 1.0000, Rappel = 1.0000
Classe 1 (2): Précision = 0.7727, Rappel = 1.0000
Classe 2 (3): Précision = 1.0000, Rappel = 0.7059


In [10]:
# 5. Implémentation d'un Perceptron Multi-couches (MLP)

print("5. PERCEPTRON MULTI-COUCHES (MLP)")
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# SANS normalisation
print("\n MLP SANS NORMALISATION ")
mlp_sans_norm = MLPClassifier(
    hidden_layer_sizes=(12,10,16 ),      
    activation='relu',            
    solver='adam',                
    learning_rate_init=0.01,     
    max_iter=5000,                
    random_state=42              
        
)

mlp_sans_norm.fit(X_train, y_train)
print(f"Nombre d'itérations: {mlp_sans_norm.n_iter_}")
print(f"Loss final: {mlp_sans_norm.loss_:.6f}")


# AVEC normalisation
print("\n MLP AVEC NORMALISATION")
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

mlp_avec_norm = MLPClassifier(
    hidden_layer_sizes=(12,10,10 ),
    activation='relu',
    solver='adam',
    learning_rate_init=0.01,
    max_iter=5000,                
    random_state=42
      
)

mlp_avec_norm.fit(X_train_normalized, y_train)
print(f"Nombre d'itérations: {mlp_avec_norm.n_iter_}")
print(f"Loss final: {mlp_avec_norm.loss_:.6f}")


5. PERCEPTRON MULTI-COUCHES (MLP)

 MLP SANS NORMALISATION 
Nombre d'itérations: 247
Loss final: 0.011412

 MLP AVEC NORMALISATION
Nombre d'itérations: 132
Loss final: 0.003600


In [ ]:
# 6. Évaluation des performances du modèle MLP

print("6. ÉVALUATION DU MLP")

# Prédictions SANS normalisation
print("\n RÉSULTATS SANS NORMALISATION ")
y_pred_mlp_sans_norm = mlp_sans_norm.predict(X_test)

cm_mlp_sans, acc_mlp_sans, prec_mlp_sans, rec_mlp_sans = compute_metrics(
    y_test, y_pred_mlp_sans_norm, n_classes
)

print("\nMatrice de Confusion:")
print(cm_mlp_sans)
print(f"\nAccuracy globale: {acc_mlp_sans:.4f} ({acc_mlp_sans*100:.2f}%)")
print("\nMétriques par classe:")
for i in range(n_classes):
    class_name = label_encoder.classes_[i]
    print(f"Classe {i} ({class_name}): Précision = {prec_mlp_sans[i]:.4f}, Rappel = {rec_mlp_sans[i]:.4f}")

# Prédictions AVEC normalisation
print("\n RÉSULTATS AVEC NORMALISATION ")
y_pred_mlp_avec_norm = mlp_avec_norm.predict(X_test_normalized)

cm_mlp_avec, acc_mlp_avec, prec_mlp_avec, rec_mlp_avec = compute_metrics(
    y_test, y_pred_mlp_avec_norm, n_classes
)

print("\nMatrice de Confusion:")
print(cm_mlp_avec)
print(f"\nAccuracy globale: {acc_mlp_avec:.4f} ({acc_mlp_avec*100:.2f}%)")
print("\nMétriques par classe:")
for i in range(n_classes):
    class_name = label_encoder.classes_[i]
    print(f"Classe {i} ({class_name}): Précision = {prec_mlp_avec[i]:.4f}, Rappel = {rec_mlp_avec[i]:.4f}")

# COMPARAISON FINALE 

print("TABLEAU COMPARATIF DES PERFORMANCES")
results = [
    ("Perceptron", acc_perceptron, "-", "-"),
    ("MLP sans normalisation", acc_mlp_sans, mlp_sans_norm.n_iter_, mlp_sans_norm.loss_),
    ("MLP avec normalisation", acc_mlp_avec, mlp_avec_norm.n_iter_, mlp_avec_norm.loss_)
]

print(f"\n{'Modèle':<30} {'Accuracy':<12} {'Iter':<8} {'Loss':<10}")
for name, acc, iter_count, loss in results:
    if iter_count == "-":
        print(f"{name:<30} {acc:.4f} ({acc*100:5.2f}%)")
    else:
        print(f"{name:<30} {acc:.4f} ({acc*100:5.2f}%)  {iter_count:<8} {loss:.6f}")




6. ÉVALUATION DU MLP

 RÉSULTATS SANS NORMALISATION 

Matrice de Confusion:
[[16  0  0]
 [ 0 17  0]
 [ 0  2 15]]

Accuracy globale: 0.9600 (96.00%)

Métriques par classe:
Classe 0 (1): Précision = 1.0000, Rappel = 1.0000
Classe 1 (2): Précision = 0.8947, Rappel = 1.0000
Classe 2 (3): Précision = 1.0000, Rappel = 0.8824

 RÉSULTATS AVEC NORMALISATION 

Matrice de Confusion:
[[15  1  0]
 [ 0 17  0]
 [ 0  4 13]]

Accuracy globale: 0.9000 (90.00%)

Métriques par classe:
Classe 0 (1): Précision = 1.0000, Rappel = 0.9375
Classe 1 (2): Précision = 0.7727, Rappel = 1.0000
Classe 2 (3): Précision = 1.0000, Rappel = 0.7647
TABLEAU COMPARATIF DES PERFORMANCES

Modèle                         Accuracy     Iter     Loss      
Perceptron                     0.9000 (90.00%)
MLP sans normalisation         0.9600 (96.00%)  247      0.011412
MLP avec normalisation         0.9000 (90.00%)  132      0.003600


"Interprétation:\n",
    "\n",
    "On constate que notre Perceptron ayant 16 neurones performe bien : il prédit avec une accuracy de 90%, avec une précision et un rappel parfait pour la première classe, un rappel parfait pour la 2ème et une précision parfaite pour la 3ème classe. On observe qu'il a malgré tout du mal avec la classe 3.\n",
    "\n",
    "Quant à notre MLP, bien que l'accuracy est plus élevée dans celui avec normalisation de données (96% contre 90% avec normalisation), on remarque toutefois que la loss a diminué drastiquement, elle passe de 0.01 a 0.003 pour les données normalisées. La convergence a été beaucoup plus rapide également."
   

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# 7. Bagging de réseaux de neurones

print("7. BAGGING DE RÉSEAUX DE NEURONES")


K = 10  # Nombre de classifieurs
modeles_bagging = []


print(f"Entraînement de {K} modèles MLP sur échantillons bootstrap...")

for i in range(K):
    # tirage avec remise
    indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
    X_bootstrap = X_train[indices]
    y_bootstrap = y_train[indices]
    
    
    mlp = MLPClassifier(
        hidden_layer_sizes=(3,), 
        max_iter=5000, 
        random_state=i 
    )
    mlp.fit(X_bootstrap, y_bootstrap)
    modeles_bagging.append(mlp)

# (Vote Majoritaire) 
def predict_bagging(models, X):
    
    predictions = np.array([m.predict(X) for m in models])
    # Vote majoritaire pour chaque échantillon
    from scipy.stats import mode
    y_final, _ = mode(predictions, axis=0)
    return y_final.flatten()


y_pred_bagging = predict_bagging(modeles_bagging, X_test)
acc_bagging = accuracy_score(y_test, y_pred_bagging)

print(f"\nAccuracy du Bagging (K={K}) : {acc_bagging:.4f} ({acc_bagging*100:.2f}%)")
print(f"Comparaison avec MLP seul : {acc_mlp_sans:.4f}")

7. BAGGING DE RÉSEAUX DE NEURONES
Entraînement de 10 modèles MLP sur échantillons bootstrap...

Accuracy du Bagging (K=10) : 1.0000 (100.00%)
Comparaison avec MLP seul : 0.9600


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import mode

#Pipeline pour lancer sur les fichiers
def executer_pipeline_final(file_name):
    print(f"ANALYSE DE : {file_name}")
    
    try:
        
        df = None
        for s in ['\t', ',', '\s+']: 
            try:
                temp_df = pd.read_csv(file_name, sep=s, header=None, engine='python')
                if temp_df.shape[1] > 1:
                    df = temp_df
                    break
            except:
                continue
        
        if df is None:
            print(f"Impossible de lire {file_name}")
            return

        # Séparation X et y
        X = df.iloc[:, :-1].values
        y = LabelEncoder().fit_transform(df.iloc[:, -1].values)
        
        # Découpage 2/3 - 1/3 
        try:
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=1/3, random_state=42, stratify=y
            )
        except:
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=1/3, random_state=42
            )

        # Normalisation
        scaler = StandardScaler()
        X_train_norm = scaler.fit_transform(X_train)
        X_test_norm = scaler.transform(X_test)

        # A. PERCEPTRON
     
        perc = MultiClassPerceptron(learning_rate=0.01, max_iter=1000)
        perc.fit(X_train, y_train)
        y_pred_p = perc.predict(X_test)
        acc_p = accuracy_score(y_test, y_pred_p)

        # B. MLP (Scikit-learn)
        mlp = MLPClassifier(hidden_layer_sizes=(12, 10, 10), max_iter=5000, random_state=42)
        mlp.fit(X_train_norm, y_train)
        acc_mlp = accuracy_score(y_test, mlp.predict(X_test_norm))

        # C. BAGGING (K=10)
        K = 10
        models = []
        for i in range(K):
            idx = np.random.choice(len(X_train_norm), size=len(X_train_norm), replace=True)
            m = MLPClassifier(hidden_layer_sizes=(3,), max_iter=5000, random_state=i)
            m.fit(X_train_norm[idx], y_train[idx])
            models.append(m)
        
        preds = np.array([mod.predict(X_test_norm) for mod in models])
        y_final, _ = mode(preds, axis=0)
        acc_bag = accuracy_score(y_test, y_final.flatten())

        print(f"-> Perceptron : {acc_p*100:.2f}%")
        print(f"-> MLP Seul   : {acc_mlp*100:.2f}%")
        print(f"-> Bagging    : {acc_bag*100:.2f}%")

    except Exception as e:
        print(f"Erreur sur {file_name} : {e}")

#Fichiers
fichiers = ['iris.txt', 'glass.txt', 'breast-cancer-wisconsin.txt', 'Lsun.txt', 'Wave.txt']
for f in fichiers:
    executer_pipeline_final(f)

ANALYSE DE : iris.txt
-> Perceptron : 90.00%
-> MLP Seul   : 92.00%
-> Bagging    : 96.00%
ANALYSE DE : glass.txt
-> Perceptron : 47.22%
-> MLP Seul   : 80.56%
-> Bagging    : 61.11%
ANALYSE DE : breast-cancer-wisconsin.txt
-> Perceptron : 95.71%
-> MLP Seul   : 94.85%
-> Bagging    : 96.14%
ANALYSE DE : Lsun.txt
-> Perceptron : 100.00%
-> MLP Seul   : 100.00%
-> Bagging    : 100.00%
ANALYSE DE : Wave.txt
-> Perceptron : 81.64%
-> MLP Seul   : 83.56%
-> Bagging    : 86.92%
